In [134]:
from __future__ import print_function
import os.path
import dalmatian as dm
import pandas as pd
import sys
pathtoJK = "../JKBio"
sys.path.insert(0, pathtoJK)
import TerraFunction as terra
import CCLF_processing as cclf

from Helper import *
import numpy as np
from gsheets import Sheets
# https://github.com/jkobject/JKBIO

"""
Log into the Google Developers Console with the Google account whose spreadsheets you want to access.
Create (or select) a project and enable the Drive API and Sheets API (under Google Apps APIs).

https://console.developers.google.com/

Go to the Credentials for your project and create New credentials > OAuth client ID > of type Other.
In the list of your OAuth 2.0 client IDs click Download JSON for the Client ID you just created.
Save the file as client_secrets.json in your home directory (user directory).
Another file, named storage.json in this example, will be created after successful authorization
to cache OAuth data.

On you first usage of gsheets with this file (holding the client secrets),
your webbrowser will be opened, asking you to log in with your Google account to authorize
this client read access to all its Google Drive files and Google Sheets.
"""
sheets = Sheets.from_files('~/.client_secret.json', '~/.storage.json')
replace = {'T': 'Tumor', 'N': 'Normal', 'm': 'Unknown', 'L': 'Unknown'}

# Initialization

In [135]:
samplesetname="CCLF_TWIST1-4"
date="2019"
data_namespace="broad-genomics-delivery"
data_workspace="Cancer_Cell_Line_Factory_CCLF_PanCancer_PanelSeq"
proc_namespace="nci-mimoun-bi-org"
proc_workspace="CCLF_Targeted"
source="CCLF"
site="HT33MBCX2"
tsca_id="TWIST1-4"
TSCA_version="TWISTv1"
picard_aggregation_type_validation="PCR"
forcekeep=[]
cohorts2id="https://docs.google.com/spreadsheets/d/1R97pgzoX0YClGDr5nmQYQwimnKXxDBGnGzg7YPlhZJU"
gsheeturllist = ["https://docs.google.com/spreadsheets/d/1LR8OFylVClxf0kmZpAdlVjrn3RBcfZKpNoDYtKdnHB8",
"https://docs.google.com/spreadsheets/d/128dkFhL1A0GqTjmR7iMvBZE8j6ymO8krBL9WX-wUAn4",
"https://docs.google.com/spreadsheets/d/1S3DqBdVkd9dLP1PDYcdSWuD2Iy2gJpzuYBhvmP37UxU",
"https://docs.google.com/spreadsheets/d/1kVIeIw66AxWLhAZlqUnAY17S87Rtfhijf1o3x0hG3Jw"]
wfrom = dm.WorkspaceManager(data_namespace, data_workspace)
wmto = dm.WorkspaceManager(proc_namespace, proc_workspace)

# Getting the samples

- we load the samples from data workspace and load the metadata files
- we remove data that has already been processed
- we create the final ids

In [172]:
# we look at all the samples we already have
refsamples = wto.get_samples()
refids = refsamples.index

In [148]:
gsheets = [sheets.get(url).sheets[0].to_frame() for url in gsheeturllist]

2019-10-07 10:55:15::WARNING  file_cache is unavailable when using oauth2client >= 4.0.0 or google-auth
Traceback (most recent call last):
  File "/anaconda3/envs/py36/lib/python3.6/site-packages/googleapiclient/discovery_cache/__init__.py", line 36, in autodetect
    from google.appengine.api import memcache
ModuleNotFoundError: No module named 'google.appengine'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/anaconda3/envs/py36/lib/python3.6/site-packages/googleapiclient/discovery_cache/file_cache.py", line 33, in <module>
    from oauth2client.contrib.locked_file import LockedFile
ModuleNotFoundError: No module named 'oauth2client.contrib.locked_file'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/anaconda3/envs/py36/lib/python3.6/site-packages/googleapiclient/discovery_cache/file_cache.py", line 37, in <module>
    from oauth2client.locked_file impor

In [181]:
metadata = pd.concat(gsheets,sort=False)
len(metadata)

192

In [182]:
# we look at all the samples we already have
cohorts = sheets.get(cohorts2id).sheets[0].to_frame()
# we use this gsheet package to get all the sheets into one dataframe

# we do some corrections just in case
samples1 = wfrom.get_samples().replace(np.nan, '', regex=True)

# creating sample_id (like in processing workspace) for metadata and samples1
newmetadata = metadata.dropna(0, subset=['Collaborator Sample ID','Sample Type','Exported DNA SM-ID']) 
print("dropped "+str(set(metadata.index.tolist())-set(newmetadata.index.tolist())))
print('lenght: '+str(len(newmetadata)))
metadata=newmetadata

2019-10-07 11:08:42::INFO  URL being requested: GET https://sheets.googleapis.com/v4/spreadsheets/1R97pgzoX0YClGDr5nmQYQwimnKXxDBGnGzg7YPlhZJU?alt=json
2019-10-07 11:08:42::INFO  URL being requested: GET https://sheets.googleapis.com/v4/spreadsheets/1R97pgzoX0YClGDr5nmQYQwimnKXxDBGnGzg7YPlhZJU/values:batchGet?majorDimension=ROWS&valueRenderOption=UNFORMATTED_VALUE&dateTimeRenderOption=FORMATTED_STRING&ranges=0&alt=json


dropped set()
lenght: 190


In [183]:
ttype = [i for i in metadata["Sample Type"]]
metadata['sample_id'] = [str(val['Collaborator Sample ID'][:-1]) + '-' + str(val['Sample Type']) + '-' + str(val['Exported DNA SM-ID']) for i, val in metadata.iterrows()]

samples1.index = [i.split('_')[2] for i, val in samples1.iterrows()]

samples1['sample_id'] = [str(val["individual_alias"]) + '-' + str(val['sample_type']) + '-' + i for i, val in samples1.iterrows()]
metadata.index = metadata['Exported DNA SM-ID']

In [184]:
# filtering on what already exists in the processing workspace (refids)
newsamples = samples1[(~samples1.index.isin(refids)) | samples1.index.isin(forcekeep)]
tokeep = set(metadata.index) & set(newsamples.index)
tokeep

{'SM-IF6GG',
 'SM-IF6GH',
 'SM-IF6GI',
 'SM-IF6GJ',
 'SM-IF6GK',
 'SM-IF6GR',
 'SM-IF6GS',
 'SM-IF6GT',
 'SM-IF6GU',
 'SM-IF6GV',
 'SM-IF6GW',
 'SM-IF6H4',
 'SM-IF6H5',
 'SM-IF6H6',
 'SM-IF6H7',
 'SM-IF6H8',
 'SM-IF6H9',
 'SM-IF6HG',
 'SM-IF6HH',
 'SM-IF6HI',
 'SM-IF6HJ',
 'SM-IF6HK',
 'SM-IF6HL',
 'SM-IF6HS',
 'SM-IF6HT',
 'SM-IF6HU',
 'SM-IF6HV',
 'SM-IF6HW',
 'SM-IF6HX',
 'SM-IF6I5',
 'SM-IF6I6',
 'SM-IF6I7',
 'SM-IF6I8',
 'SM-IF6I9',
 'SM-IF6IA',
 'SM-IF6IH',
 'SM-IF6II',
 'SM-IF6IJ',
 'SM-IF6IK',
 'SM-IF6IL',
 'SM-IF6IM',
 'SM-IF6IT',
 'SM-IF6IU',
 'SM-IF6IV',
 'SM-IF6IW',
 'SM-IF6IX',
 'SM-IF6IY',
 'SM-J1OYQ',
 'SM-J1OYR',
 'SM-J1OYS',
 'SM-J1OYT',
 'SM-J1OYU',
 'SM-J1OYV',
 'SM-J1OYW',
 'SM-J1OYX',
 'SM-J1OYY',
 'SM-J1OYZ',
 'SM-J1OZ1',
 'SM-J1OZ2',
 'SM-J1OZ3',
 'SM-J1OZ4',
 'SM-J1OZ5',
 'SM-J1OZ7',
 'SM-J1OZ8',
 'SM-J1OZ9',
 'SM-J1OZA',
 'SM-J1OZB',
 'SM-J1OZC',
 'SM-J1OZD',
 'SM-J1OZE',
 'SM-J1OZF',
 'SM-J1OZG',
 'SM-J1OZH',
 'SM-J1OZI',
 'SM-J1OZJ',
 'SM-J1OZK',
 'SM-J1OZL',

In [185]:
# usefull to merge the two df, sm-id is one of the only unique id here
if len(newsamples[~newsamples.index.isin(tokeep)]) > 0:
    print('we could not add these as we dont have metadata for them:' + str(newsamples[~newsamples.index.isin(tokeep)].index))
newsamples = newsamples[newsamples.index.isin(tokeep)]
newmetadata = metadata[metadata.index.isin(tokeep)].sort_index().drop_duplicates("Exported DNA SM-ID")

we could not add these as we dont have metadata for them:Index(['LCSET-15778', 'LCSET-16192', 'LCSET-16427', 'SM-JD9WI', 'SM-JD9WJ',
       'SM-JD9WK', 'SM-JD9WL', 'SM-JD9WM', 'SM-JD9WN', 'SM-JD9WO', 'SM-JD9WP',
       'SM-JD9WQ', 'SM-JD9WR', 'SM-JD9WS', 'SM-JD9WT', 'SM-JD9WU', 'SM-JD9WV',
       'SM-JD9WW', 'SM-JD9WX', 'SM-JD9WY', 'SM-JD9WZ', 'SM-JD9X1', 'SM-JD9X2',
       'SM-JD9X3', 'SM-JD9X4', 'SM-JD9X5', 'SM-JD9X6', 'SM-JD9X7', 'SM-JD9X8',
       'SM-JD9X9', 'SM-JD9XA', 'SM-JD9XB', 'SM-JD9XC', 'SM-JD9XD', 'SM-JD9XE',
       'SM-JD9XF', 'SM-JD9XG', 'SM-JD9XH', 'SM-JD9XI', 'SM-JD9XJ', 'SM-JD9XK',
       'SM-JD9XL', 'SM-JD9XM', 'SM-JD9XN', 'SM-JD9XO', 'SM-JD9XP', 'SM-JD9XQ',
       'SM-JD9XR'],
      dtype='object')


In [119]:
newsamples.shape

(140, 222)

# Creating the sample information dataframe

In [210]:
print('we are dropping '+str(df.iloc[[j for j,i in enumerate(df[['Collaborator Participant ID','Exported DNA SM-ID','Stock DNA SM-ID','Sample Type','Tumor Type','Original Material Type', 'Material Type','Primary Disease','Media on Tube','Collection','Tissue Site']].isna().values.sum(1)) if i !=0]].index.tolist()) +' as they dont have full data')
df = df.iloc[[j for j,i in enumerate(df[['Exported DNA SM-ID','Collaborator Participant ID','Stock DNA SM-ID','Sample Type','Tumor Type','Original Material Type', 'Material Type','Primary Disease','Media on Tube','Collection','Tissue Site']].isna().values.sum(1)) if i ==0]]

we are dropping ['SM-IF6GR', 'SM-IF6H4', 'SM-IF6HG', 'SM-IF6HH', 'SM-IF6HS', 'SM-IF6I5', 'SM-IF6IH', 'SM-IF6IT', 'SM-IF6IV', 'SM-IF6IY', 'SM-J1OYQ', 'SM-J1OYR', 'SM-J1OYT', 'SM-J1OYU', 'SM-J1OYV', 'SM-J1OYW', 'SM-J1OYX', 'SM-J1OYY', 'SM-J1OZ1', 'SM-J1OZ3', 'SM-J1OZ4', 'SM-J1OZ5', 'SM-J1OZ7', 'SM-J1OZ8', 'SM-J1OZA', 'SM-J1OZB', 'SM-J1OZC', 'SM-J1OZD', 'SM-J1OZE', 'SM-J1OZF', 'SM-J1OZG', 'SM-J1OZH', 'SM-J1OZI', 'SM-J1OZJ', 'SM-J1OZK', 'SM-J1OZL', 'SM-J1OZM', 'SM-J1OZN', 'SM-J1OZO', 'SM-J1OZP', 'SM-J1OZQ', 'SM-J1OZR', 'SM-J1OZS', 'SM-J1OZT', 'SM-J1OZU', 'SM-J1OZV', 'SM-J1OZW', 'SM-J1OZX', 'SM-J1OZY', 'SM-J1P11', 'SM-J1P12', 'SM-J8MCA', 'SM-J8MCB', 'SM-J8MCC', 'SM-J8MCD', 'SM-J8MCE', 'SM-J8MCF', 'SM-J8MCI', 'SM-J8MCJ', 'SM-J8MCK', 'SM-J8MCL', 'SM-J8MCM', 'SM-J8MCN', 'SM-J8MCP', 'SM-J8MCQ', 'SM-J8MCR', 'SM-J8MCS', 'SM-J8MCT', 'SM-J8MCU', 'SM-J8MCW', 'SM-J8MCX', 'SM-J8MCY', 'SM-J8MCZ', 'SM-J8MD1', 'SM-J8MD2', 'SM-J8MD4', 'SM-J8MD5', 'SM-J8MD6', 'SM-J8MD7', 'SM-J8MD8', 'SM-J8MD9', 'SM-J8MDB',

In [186]:
print('creating new df')
df = pd.concat([newmetadata, newsamples], axis=1, sort=True)
# from this new set we create a dataframe which will get uploaded to terra
sample_info = df[['crai_or_bai_path', 'cram_or_bam_path']]
sample_info['individual_id'] = df['Collaborator Participant ID'].astype(str)
sample_info['reference_id'] = df['Exported DNA SM-ID'].astype(str)
sample_info['participant'] = df['Collaborator Participant ID'].astype(str)
sample_info['aggregation_product_name_validation'] = [TSCA_version] * sample_info.shape[0]
# here we add this number as the reference id might be present many times already for different samples
# in the processing workspace
sample_info['external_id_validation'] = [i +'_'+ str(refsamples[refsamples['external_id_validation'] == i].shape[1]) if refsamples[refsamples['external_id_validation'] == i].shape[0] > 0 else i for i in sample_info['reference_id']]
sample_info['bsp_sample_id_validation'] = df.index.astype(str)
sample_info['stock_sample_id_validation'] = df['Stock DNA SM-ID'].astype(str)
sample_info['sample_type'] = df['Sample Type']
sample_info['picard_aggregation_type_validation'] = [picard_aggregation_type_validation] * sample_info.shape[0]
sample_info['tumor_subtype'] = df['Tumor Type'].astype(str)
sample_info['squid_sample_id_validation'] = sample_info['external_id_validation']
sample_info['source_subtype_validation'] = df['Original Material Type'].astype(str)
sample_info['processed_subtype_validation'] = df['Material Type'].astype(str)
sample_info['primary_disease'] = df['Primary Disease'].astype(str)
sample_info['media'] = df['Media on Tube'].astype(str)
sample_info['Collection'] = df['Collection'].astype(str)
# match collection data and error out
cohortlist = []
for k, val in sample_info['Collection'].iteritems():
    res = cohorts[cohorts['Name'] == val]
    if len(res) == 0:
        print("we do not have a correponsding cohort for this collection for sample: "+str(k))
        cohortlist.append('nan')
    else:
        cohortlist.append(res['ID'].values[0])
sample_info['cohorts'] = cohortlist

sample_info['tissue_site'] = df['Tissue Site'].astype(str)
sample_info['source'] = [source] * sample_info.shape[0]
sample_info['sample_id'] = df.index.astype(str)

sample_info = sample_info.set_index('sample_id')

creating new df


/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/sta

we do not have a correponsding cohort for this collection for sample: SM-IF6GR
we do not have a correponsding cohort for this collection for sample: SM-IF6H4
we do not have a correponsding cohort for this collection for sample: SM-IF6HG
we do not have a correponsding cohort for this collection for sample: SM-IF6HS
we do not have a correponsding cohort for this collection for sample: SM-IF6I5


/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/in

In [190]:
sample_info.isna()

,crai_or_bai_path,cram_or_bam_path,individual_id,reference_id,participant,aggregation_product_name_validation,external_id_validation,bsp_sample_id_validation,stock_sample_id_validation,sample_type,...,tumor_subtype,squid_sample_id_validation,source_subtype_validation,processed_subtype_validation,primary_disease,media,Collection,cohorts,tissue_site,source
sample_id,,,,,,,,,,,,,,,,,,,,,
SM-IF6GG,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
SM-IF6GH,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
SM-IF6GI,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
SM-IF6GJ,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
SM-IF6GK,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
SM-IF6GR,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
SM-IF6GS,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
SM-IF6GT,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
SM-IF6GU,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


# Creating the sample_sets

In [159]:
normals = [r["participant"] for i, r in sample_info.iterrows() if r['sample_type'] == "Normal"]
normalsid = [i for i, r in sample_info.iterrows() if r['sample_type'] == "Normal"]
tumors = [r["participant"] for i, r in sample_info.iterrows() if r['sample_type'] == "Tumor"]
tumorsid = [i for i, r in sample_info.iterrows() if r['sample_type'] == "Tumor"]
prevtumors = [val["participant"] for k, val in refsamples.iterrows() if val.sample_type == "Tumor"]
prevnormals = [val["participant"] for k, val in refsamples.iterrows() if val.sample_type == "Normal"]

print("creating new pairs")
# do we have new tumors/normals for our previous ones
newpairs = {'pair_id': [], 'case_sample': [], 'control_sample': [], 'participant': [], 'match_type':[]}

toreprocess_normals = set(tumors) & set(prevnormals)
for val in toreprocess_normals:
    if val != 'nan':
        for tumor_id in sample_info[sample_info['participant'] == val][sample_info[
                'sample_type'] == 'Tumor'].index.tolist():
            normal_id = refsamples[refsamples['participant'] == val][refsamples[
              'sample_type'] == 'Normal'].index.tolist()[0]
            newpairs['pair_id'].append(tumor_id + '_' + normal_id)
            newpairs['case_sample'].append(tumor_id)
            newpairs['control_sample'].append(normal_id)
            newpairs['participant'].append(val)
            newpairs['match_type'].append("Tumor_Normal")

paired = set(tumors) & set(normals)
for val in set(tumors) - toreprocess_normals:
    if val != 'nan':
        for tumor_id in sample_info[sample_info['participant'] == val][sample_info[
                'sample_type'] == 'Tumor'].index.tolist():
            normal_id = sample_info[(sample_info['participant'] == val) & (sample_info[
              'sample_type'] == 'Normal')].index.tolist()[0] if val in paired else 'NA'
            newpairs['pair_id'].append(tumor_id + "_" + normal_id)
            newpairs['case_sample'].append(tumor_id)
            newpairs['control_sample'].append(normal_id)
            newpairs['participant'].append(val)
            newpairs['match_type'].append("Tumor_Normal" if val in paired else 'Tumor_NA')

newpairs = pd.DataFrame(newpairs).set_index('pair_id')

creating new pairs


/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:15: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  from ipykernel import kernelapp as app
/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:17: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:26: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


# Uploads to Terra

## all the entities need to exist! Else it will raise an error and block further uploads to Terra

In [160]:
print("all the entities need to exist! Else it will raise an error and block further uploads to Terra")
print("uploading new samples")
wto.upload_samples(sample_info)
if not "NA" in wto.get_samples().index.tolist():
    wto.upload_samples(pd.DataFrame({'sample_id':['NA'], 'participant_id':['NA']}).set_index('sample_id'))
    
print("creating pairs and pairsets")
wto.upload_entities('pair', newpairs)
wto.update_pair_set(samplesetname+'_pairs', newpairs.index.tolist())
cohorts_in_batch = []
cohorts_with_pairs = []
for val in cohorts['ID'].values:
    cohortsamples=sample_info[sample_info["cohorts"] == val].index.tolist()
    tumorsamplesincohort = sample_info[sample_info["cohorts"] == val][sample_info['sample_type']=="Tumor"].index.tolist()
    pairsamples=newpairs[newpairs['case_sample'].isin(tumorsamplesincohort)].index.tolist()
    if len(cohortsamples)>0:
        cohorts_in_batch.append(val)
        try:
            terra.addToSampleSet(wto, val, cohortsamples)
        except KeyError: # we may not have this set yet
            wto.update_sample_set(val, cohortsamples)
    if len(pairsamples)>0:
        cohorts_with_pairs.append(val)
        try:
            terra.addToPairSet(wto,val, pairsamples)
        except KeyError: # we may not have this set yet
            wto.update_pair_set(val, pairsamples)
print("creating a sample set")
wto.update_sample_set(sample_set_id=samplesetname + "_all", sample_ids=sample_info.index.tolist())
wto.update_sample_set(sample_set_id=samplesetname + "_tumors", sample_ids=tumorsid)
wto.update_sample_set(sample_set_id=samplesetname + "_normals", sample_ids=normalsid)
normalsid.extend([k for k, val in refsamples.iterrows() if val.sample_type == "Normal"])
# Same as cum pon but better
terra.addToSampleSet(sample_set_id="All_normals_TWIST", sample_ids=normalsid)
all_samples = wto.get_samples().index.tolist()
all_samples.remove('NA')
terra.addToSampleSet(sample_set_id="All_samples_TWIST", sample_ids=all_samples)

all the entities need to exist! Else it will raise an error and block further uploads to Terra
uploading new samples
Successfully imported 49 participants.


/anaconda3/envs/py36/lib/python3.6/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


Successfully imported 188 samples.
Updating many hound records. Switching to batch updates
Hound executing batch upload of 8274 records
creating pairs and pairsets
Successfully imported 113 pairs.
Updating many hound records. Switching to batch updates
Hound executing batch upload of 906 records
Successfully imported 1 pair sets:
  * trial1_pairs (113 pairs)
Updating many hound records. Switching to batch updates
Hound executing batch upload of 2 records


TypeError: Object of type 'Index' is not JSON serializable

# Running Terra Worlflows

In [163]:
print("Creating Terra submissions: remember you can only cancel \
    or interact with terra submissions from the Terra website. \
    https://app.terra.bio/#workspaces/"+proc_namespace.replace(" ", "%20")+"/"+proc_workspace.replace(" ", "%20")+"/job_history")

RenameBAM_TWIST = wto.create_submission("RenameBAM_TWIST", samplesetname + "_all", 'sample_set', expression='this.samples')
print("waiting for 'Rename'")
terra.waitForSubmission(wto, [RenameBAM_TWIST])

Creating Terra submissions: remember you can only cancel     or interact with terra submissions from the Terra website.     https://app.terra.bio/#workspaces/nci-mimoun-bi-org/CCLF_Targeted/job_history
Successfully created submission f1f793c4-be79-44bd-a3ca-52528d6afe0a.
waiting for 'Rename'
1.0 of jobs Succeeded in submission 0.sion 0. 30 mn elapsed.d.


[]

In [165]:
CalculateTargetCoverage_PANCAN = wto.create_submission('CalculateTargetCoverage', samplesetname + "_all", 'sample_set', expression='this.samples')
DepthOfCov_PANCAN = wto.create_submission('DepthOfCov_PANCAN', samplesetname + "_all", 'sample_set', expression='this.samples')
print("waiting for 'CalculateTargetCoverage' & 'DepthOfCov_PANCAN'")
terra.waitForSubmission(wto, [CalculateTargetCoverage_PANCAN, DepthOfCov_PANCAN])

Successfully created submission 048b8d2c-ba6c-45d1-9133-abeeaf39de18.
Successfully created submission d1660eae-c83a-4088-bd79-1265bd2761e0.
waiting for 'CalculateTargetCoverage' & 'DepthOfCov_PANCAN'
1.0 of jobs Succeeded in submission 0.sion 0. 15 mn elapsed.d.
1.0 of jobs Succeeded in submission 1.sion 1. 70 mn elapsed.d.


[]

In [189]:
CreatePanelOfNormalsGATK_PANCAN = wto.create_submission('CreatePanelOfNormalsGATK_PANCAN', 'All_normals_TWIST')
DepthOfCovQC_PANCAN = wto.create_submission('DepthOfCovQC_PANCAN', samplesetname + "_all", 'sample_set', expression='this.samples')
print("waiting for 'DepthOfCovQC_PANCAN' & 'CNV_CreatePoNForCNV'")
terra.waitForSubmission(wto, [DepthOfCovQC_PANCAN, CreatePanelOfNormalsGATK_PANCAN])

Successfully created submission da72cd1f-d357-4cc7-a8a2-45fcae7c12e4.


ValueError: No such sample_set 'CCLF_TWIST1-4_all' in this workspace. Check your entity and entity type

In [191]:
DepthOfCovQC_PANCAN = wto.create_submission('DepthOfCovQC_PANCAN', samplesetname + "_all", 'sample_set', expression='this.samples')
print("waiting for 'DepthOfCovQC_PANCAN' & 'CNV_CreatePoNForCNV'")
terra.waitForSubmission(wto, [DepthOfCovQC_PANCAN, CreatePanelOfNormalsGATK_PANCAN])

Successfully created submission 0a3d8be8-ccbb-4b97-9266-de256fcd9ce6.
waiting for 'DepthOfCovQC_PANCAN' & 'CNV_CreatePoNForCNV'
1.0 of jobs Succeeded in submission 0.sion 0. 6 mn elapsed.d.
1.0 of jobs Succeeded in submission 1.


[]

In [193]:
CallSomaticCNV_PANCAN = wto.create_submission('CallSomaticCNV_PANCAN', samplesetname + "_all", 'sample_set', expression='this.samples')
print("waiting for 'CallSomaticCNV_PANCAN'")
terra.waitForSubmission(wto, [CallSomaticCNV_PANCAN])

Successfully created submission 6f048b10-71f5-4b0c-8637-a22c3f394016.
waiting for 'CallSomaticCNV_PANCAN'
1.0 of jobs Succeeded in submission 0.sion 0. 12 mn elapsed.d.


[]

In [194]:
MutationCalling_Normals_TWIST = wto.create_submission("MutationCalling_Normals_TWIST", samplesetname + "_normals", 'sample_set', expression='this.samples')
print("waiting for 'MutationCalling_Normals_TWIST'")
terra.waitForSubmission(wto, [MutationCalling_Normals_TWIST])

Successfully created submission 51583424-59a3-4db0-a5d2-beb1b489acb6.
waiting for 'MutationCalling_Normals_TWIST'


APIException: (502) : 
<html><head>
<meta http-equiv="content-type" content="text/html;charset=utf-8">
<title>502 Server Error</title>
</head>
<body text=#000000 bgcolor=#ffffff>
<h1>Error: Server Error</h1>
<h2>The server encountered a temporary error and could not complete your request.<p>Please try again in 30 seconds.</h2>
<h2></h2>
</body></html>


In [195]:
FilterGermlineVariants_NormalSample_TWIST = wto.create_submission('FilterGermlineVariants_NormalSample_TWIST', samplesetname + "_normals", 'sample_set', expression='this.samples')
print("waiting for 'SNV_FilterGermline'")
terra.waitForSubmission(wto, [FilterGermlineVariants_NormalSample_TWIST])

Successfully created submission 1deab39e-e3cb-4db2-8cab-d9170b22f4e8.
waiting for 'SNV_FilterGermline'
1.0 of jobs Succeeded in submission 0.sion 0. 6 mn elapsed..


[]

In [196]:
CreatePoNSNV_Mutect1 = wto.create_submission('CreatePoNSNV_Mutect1', "All_normals_TWIST")
CreatePoN_SNV_MuTect2 = wto.create_submission('CreatePoN_SNV_MuTect2', "All_normals_TWIST")
print("waiting for 'CreatePoN_SNV_MuTect2' & 'CreatePoNSNV_Mutect1'")
terra.waitForSubmission(wto, [CreatePoNSNV_Mutect1, CreatePoN_SNV_MuTect2])

Successfully created submission 6c0f5dac-f4d4-4329-b1af-3aae08f142f7.
Successfully created submission ae2e51de-92b0-4a60-a609-12f4f738ed21.
waiting for 'CreatePoN_SNV_MuTect2' & 'CreatePoNSNV_Mutect1'
1.0 of jobs Succeeded in submission 0.sion 0. 6 mn elapsed.
1.0 of jobs Succeeded in submission 1.


[]

In [197]:
PlotSomaticCNVMaps_PANCAN = wto.create_submission('PlotSomaticCNVMaps_PANCAN', samplesetname + "_all")
for val in cohorts_in_batch:
    wto.create_submission("PlotSomaticCNVMaps_PANCAN", val)
print("submitted final jobs for CNV pipeline")

Successfully created submission b896197c-28e3-4cf3-a0a6-f7f4f768f315.
Successfully created submission dab675ed-8785-4d6c-90ee-ea868293508c.
Successfully created submission b039dd10-1c3d-4a3a-8643-3ad2ed4eb26b.
Successfully created submission b96a8cde-7b4a-4de9-949d-fa48fde9a01e.
Successfully created submission 4fb213c9-3782-4cd1-a013-1ea61b29b87e.
Successfully created submission 81241401-9f40-4634-abaf-fe44a2cc5ef9.
Successfully created submission c9730e48-5459-439c-bb67-f9df5069fe00.
Successfully created submission b7de3352-1cdd-414a-bda4-82507d1c8ce8.
Successfully created submission 9066be31-c17c-4ee5-9053-01a65d08be21.
Successfully created submission 66a6d154-47e3-46d7-aae5-0d32f0fb4f90.
Successfully created submission 8bc39869-c784-4f18-a4e3-b8496f118d06.
Successfully created submission c0295fc2-a35d-4ecb-888e-a8e0591b1ad6.
Successfully created submission ed181a5c-7d11-432c-ad11-71a534818aa7.
submitted final jobs for CNV pipeline


In [198]:
SNV_PostProcessing_Normals = wto.create_submission('SNV_PostProcessing_Normals', samplesetname + "_normals")
MutationCalling_Tumors_TWIST = wto.create_submission('MutationCalling_Tumors_TWIST', samplesetname+'_pairs', 'pair_set', expression='this.pairs')
print("waiting for 'SNV_PostProcessing' & 'MutationCalling_Tumors_TWIST'")
terra.waitForSubmission(wto, [SNV_PostProcessing_Normals, MutationCalling_Tumors_TWIST])

Successfully created submission 7f5a6f8e-f633-44ec-9ea5-74d0b3ac8dc5.
Successfully created submission e69ce0d4-d242-4305-bce8-0873329448fd.
waiting for 'SNV_PostProcessing' & 'MutationCalling_Tumors_TWIST'
1.0 of jobs Succeeded in submission 0.sion 0. 6 mn elapsed.


ConnectionError: ('Connection aborted.', OSError("(54, 'ECONNRESET')",))

In [51]:
b[b['case_sample'].isin(a)].index.tolist()

['SM-IF6GG_NA',
 'SM-IF6GH_NA',
 'SM-IF6GI_NA',
 'SM-IF6GJ_NA',
 'SM-IF6GK_NA',
 'SM-IF6GS_NA',
 'SM-IF6GT_NA',
 'SM-IF6GU_NA',
 'SM-IF6GV_NA',
 'SM-IF6GW_NA',
 'SM-IF6H5_NA',
 'SM-IF6H6_NA',
 'SM-IF6H7_NA',
 'SM-IF6H8_NA',
 'SM-IF6H9_NA',
 'SM-IF6HI_NA',
 'SM-IF6HJ_NA',
 'SM-IF6HK_NA',
 'SM-IF6HL_NA',
 'SM-IF6HT_NA',
 'SM-IF6HU_SM-HKIP5',
 'SM-IF6HV_NA',
 'SM-IF6HW_NA',
 'SM-IF6HX_NA',
 'SM-IF6I6_NA',
 'SM-IF6I7_NA',
 'SM-IF6I8_SM-HKIP5',
 'SM-IF6I9_NA',
 'SM-IF6IA_NA',
 'SM-IF6II_NA',
 'SM-IF6IJ_NA',
 'SM-IF6IK_NA',
 'SM-IF6IL_NA',
 'SM-IF6IM_NA',
 'SM-IF6IU_NA',
 'SM-IF6IV_NA',
 'SM-IF6IW_SM-HKIP5',
 'SM-IF6IX_NA',
 'SM-IF6IY_NA',
 'SM-J1OYQ_NA',
 'SM-J1OYS_NA',
 'SM-J1OYT_SM-J1OZB',
 'SM-J1OYU_NA',
 'SM-J1OYV_SM-J1OZB',
 'SM-J1OYW_SM-J1OZB',
 'SM-J1OYX_SM-J1OZB',
 'SM-J1OYY_SM-J1OZB',
 'SM-J1OYZ_NA',
 'SM-J1OZ1_SM-J1OZB',
 'SM-J1OZ2_NA',
 'SM-J1OZ3_SM-ISKDM',
 'SM-J1OZ4_NA',
 'SM-J1OZ5_NA',
 'SM-J1OZ7_SM-J1OZB',
 'SM-J1OZ8_SM-J1OZB',
 'SM-J1OZ9_NA',
 'SM-J1OZC_SM-J1OZB',
 'SM-J1OZE

In [56]:
wmto.delete_entity('pair',b[b['case_sample'].isin(a)].index.tolist())

APIException: (409) : [{"entityType":"pair","entityName":"SM-J1OYY_SM-J1OZB"},{"entityType":"pair","entityName":"SM-J8MCU_SM-J1OZB"},{"entityType":"pair","entityName":"SM-J8MD6_SM-J1OZB"},{"entityType":"pair","entityName":"SM-J1OZH_SM-J1OZB"},{"entityType":"pair","entityName":"SM-IF6IL_NA"},{"entityType":"pair","entityName":"SM-IF6IX_NA"},{"entityType":"pair","entityName":"SM-J8MCZ_SM-J1OZB"},{"entityType":"pair","entityName":"SM-J1OZP_SM-J1OZB"},{"entityType":"pair","entityName":"SM-J8MD8_SM-J1OZB"},{"entityType":"pair","entityName":"SM-IF6IU_NA"},{"entityType":"pair","entityName":"SM-J1OZC_SM-J1OZB"},{"entityType":"pair","entityName":"SM-J8MD9_SM-J1OZB"},{"entityType":"pair","entityName":"SM-J8MCX_SM-J1OZB"},{"entityType":"pair","entityName":"SM-IF6IM_NA"},{"entityType":"pair","entityName":"SM-IF6GV_NA"},{"entityType":"pair","entityName":"SM-IF6H7_NA"},{"entityType":"pair","entityName":"SM-IF6HW_NA"},{"entityType":"pair","entityName":"SM-IF6HL_NA"},{"entityType":"pair","entityName":"

In [49]:
wmto.delete_sample(a)

APIException: (409) : [{"entityType":"sample","entityName":"SM-J8MDL"},{"entityType":"sample","entityName":"SM-IF6IH"},{"entityType":"sample","entityName":"SM-J8MCK"},{"entityType":"pair","entityName":"SM-J1OYY_SM-J1OZB"},{"entityType":"sample","entityName":"SM-J1OZO"},{"entityType":"sample","entityName":"SM-J8MCZ"},{"entityType":"pair","entityName":"SM-J8MCU_SM-J1OZB"},{"entityType":"pair","entityName":"SM-J8MD6_SM-J1OZB"},{"entityType":"sample","entityName":"SM-IF6IL"},{"entityType":"pair","entityName":"SM-J1OZH_SM-J1OZB"},{"entityType":"sample","entityName":"SM-J1OZ1"},{"entityType":"pair","entityName":"SM-IF6IL_NA"},{"entityType":"sample","entityName":"SM-IF6IK"},{"entityType":"pair","entityName":"SM-IF6IX_NA"},{"entityType":"pair","entityName":"SM-J8MCZ_SM-J1OZB"},{"entityType":"pair","entityName":"SM-J1OZP_SM-J1OZB"},{"entityType":"sample","entityName":"SM-J1OZQ"},{"entityType":"pair","entityName":"SM-J8MD8_SM-J1OZB"},{"entityType":"pair","entityName":"SM-IF6IU_NA"},{"entityType"

In [7]:
## two cohorts have not worked because they contained just one acceptable cell line (the workflow needs cohorts with at least 2 acceptable CL, here both had one rejected one)
FilterGermlineEvents_TumorSample = wto.create_submission('FilterGermlineEvents_TumorSample', samplesetname+'_pairs', 'pair_set', expression='this.pairs')
print("waiting for 'FilterGermlineEvents_TumorSample'")
terra.waitForSubmission(wto, FilterGermlineEvents_TumorSample)

Successfully created submission 2e52a727-c22f-47a6-ae31-9530fbc71d2d.
waiting for 'FilterGermlineEvents_TumorSample'
1.0 of jobs Succeeded in submission 0.sion 0. 7 mn elapsed.d.


[]

In [20]:
b = wmto.get_pairs()

In [29]:
b['case_sample'].isin(a)

pair_id
SM-CJX4Z_NA_TN          False
SM-CJX4Z_NA_TP          False
SM-CJX51_NA_TN          False
SM-CJX51_NA_TP          False
SM-CJX52_NA_TN          False
SM-CJX52_NA_TP          False
SM-CJX53_NA_TN          False
SM-CJX53_NA_TP          False
SM-CJX54_NA_TN          False
SM-CJX54_NA_TP          False
SM-CNN2V_NA_TN          False
SM-CNN2V_NA_TP          False
SM-CNN37_NA_TN          False
SM-CNN37_NA_TP          False
SM-CNN38_NA_TN          False
SM-CNN38_NA_TP          False
SM-CNN4H_NA_TN          False
SM-CNN4H_NA_TP          False
SM-CNN4R_NA_TN          False
SM-CNN4R_NA_TP          False
SM-D182S_NA_TN          False
SM-D182S_NA_TP          False
SM-D182T_NA_TN          False
SM-D182T_NA_TP          False
SM-D182U_NA_TN          False
SM-D182U_NA_TP          False
SM-D182V_NA_TN          False
SM-D182V_NA_TP          False
SM-D182W_NA_TN          False
SM-D182W_NA_TP          False
                        ...  
SM-J9N86_NA_TN          False
SM-J9N86_NA_TP          False
SM

In [30]:
b.loc[b['case_sample'].isin(a),'match_type'] = ["Tumor_NA" if '_NA' in k else "Tumor_Normal" for k in b[b['case_sample'].isin(a)].index] 

Index(['SM-IF6GG_NA', 'SM-IF6GH_NA', 'SM-IF6GI_NA', 'SM-IF6GJ_NA',
       'SM-IF6GK_NA', 'SM-IF6GS_NA', 'SM-IF6GT_NA', 'SM-IF6GU_NA',
       'SM-IF6GV_NA', 'SM-IF6GW_NA',
       ...
       'SM-J8MDG_SM-J1OZB', 'SM-J8MDJ_SM-J1OZB', 'SM-J8MDK_SM-J1OZB',
       'SM-J8MDL_SM-J1OZB', 'SM-J8MDM_SM-J1OZB', 'SM-J8MDN_SM-J1OZB',
       'SM-J8MDP_SM-J1OZB', 'SM-J8MDQ_SM-J1OZB', 'SM-J8MDR_SM-J1OZB',
       'SM-J8MDT_SM-J1OZB'],
      dtype='object', name='pair_id', length=113)

In [ ]:
wmto.upload_entities('pair', )

In [8]:
SNVPostProcessing_TWIST = wto.create_submission('SNVPostProcessing_TWIST', samplesetname+'_pairs', "pair_set")
print("Submitted final jobs for SNV pipeline")

FNG_Compile_Pileup_Cnt = wto.create_submission("FNG_Compile_Pileup_Cnt", samplesetname + "_all", 'sample_set', expression='this.samples')
print("waiting for 'FNG_Compile_Pileup_Cnt'")
terra.waitForSubmission(wto, [FNG_Compile_Pileup_Cnt])

FNG_Compile_db_slow_download = wto.create_submission("FNG_Compile_db_slow_download", "All_samples")
print("waiting for 'FNG_Compile_db'")
terra.waitForSubmission(wto, [FNG_Compile_db_slow_download])

Successfully created submission 48b29bb6-96e8-4539-a6ca-80b26d3899b7.
Submitted final jobs for SNV pipeline
Successfully created submission 978a7a95-bc31-4fd3-b3c6-64b25cb46112.
waiting for 'FNG_Compile_Pileup_Cnt'
SM-IF6GUs: Failed for 0 jobs in submission 0. 11 mn elapsed.d.
SM-IF6H4s: Failed for 1 jobs in submission 0. 13 mn elapsed.d.
SM-IF6HX
0.9840425531914894 of jobs Succeeded in submission 0.


RuntimeError: 3 failed submission

In [ ]:
FNG_Query_db = wto.create_submission("FNG_Query_db", samplesetname + "_all")
print("Submitted final FNG Job")
print('Done')

## Create and upload nice folders with data per sample or per cohort or per any list provided

In [28]:
workspace1 = "CCLF_TSCA_2_0_2"
namespace1 = "nci-mimoun-bi-org"
wm1 = dm.WorkspaceManager(namespace1,workspace1)
pathto_cnvpng='segmented_copy_ratio_img'
pathto_stats='sample_statistics'
is_from_pairs=True
pathto_snv='filtered_variants'
pathto_seg='cnv_calls'
datadir='gs://cclf_results/targeted/kim_sept/'

specificlist=['CCLF_PEDS1012',
'PEDS172',
'PEDS182',
'PEDS196',
'PEDS204']

In [54]:
cclf.getReport(datadir='gs://cclf_results/targeted/kim_sept_2/',
               specificlist=specificlist)

you need to be on macOS for now
we did not found any data for PEDS182
